# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans(handle=<cuml.raft.common.handle.Handle object at 0x7f55bc2a6d50>, n_clusters=5, max_iter=300, tol=0.0001, verbose=4, random_state=1, init='scalable-k-means++', n_init=1, oversampling_factor=2.0, max_samples_per_batch=32768, output_type='input')

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.7684636 ,  2.3276033 , -3.7457771 , -1.8541752 , -5.1695833 ,
         7.667088  ,  2.7118318 ,  8.495609  ,  1.7038484 ,  1.1884266 ],
       [ 4.647688  ,  8.37788   , -9.070581  ,  9.459332  ,  8.450423  ,
        -1.0210547 ,  3.3920872 , -7.8629856 , -0.7527662 ,  0.48384127],
       [-2.9414437 ,  4.6401706 , -4.5027537 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499196 , -8.063638  ,  2.816936  ],
       [-4.271077  ,  5.561165  , -5.6640916 , -1.8229512 , -9.2925    ,
         0.73028314,  4.4586773 , -2.8876226 , -5.1257744 ,  9.694357  ],
       [ 5.5837417 , -4.1515303 ,  4.369667  , -3.0020504 ,  3.638897  ,
        -4.341912  , -3.318711  ,  6.503671  , -6.865036  , -1.0266498 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:37028 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 270.37 GB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.809875  ,  8.422671  , -9.239023  ,  9.379142  ,  8.499881  ,
        -1.0592818 ,  3.3437858 , -7.802612  , -0.5946332 ,  0.26447597,
         5.5073957 , -4.10698   ,  4.2890778 , -2.8172052 ,  3.615015  ,
        -4.1613007 , -3.6209643 ,  6.2185297 , -6.9460473 , -1.0828305 ,
        -5.82677   ,  2.2258763 , -3.8601215 , -1.6974076 , -5.313418  ,
         7.579578  ,  2.9187474 ,  8.540424  ,  1.5523204 ,  1.0841804 ],
       [-2.8941855 ,  4.4741907 , -4.447567  ,  2.3820987 ,  1.7478832 ,
        -2.5046248 , -5.208331  , -1.6937685 , -8.134755  ,  2.6468296 ,
        -4.316362  ,  5.56554   , -5.7321987 , -1.7384952 , -9.344658  ,
         0.70846575,  4.435839  , -2.9009    , -4.948638  ,  9.695301  ,
         8.366522  , -6.2474537 , -6.3494725 ,  1.9546974 ,  4.157616  ,
        -9.167902  ,  4.6070676 ,  8.788584  ,  6.8644233 ,  2.231988  ],
       [-4.6657133 , -9.558958  ,  6.657229  ,  4.440131  ,  2.1730304 ,
         2.5904036 ,  0.58000994,  6.2550354 , -8